<a href="https://colab.research.google.com/github/ashishguptap99/Speech-to-Text---Watson/blob/main/Unscripted_Ashish__STT_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Installations
!pip install ibm_watson
!pip install pydub rouge

     |████████████████████████████████| 419kB 27.8MB/s 
     |████████████████████████████████| 204kB 41.5MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-5.2.0-cp37-none-any.whl size=403252 sha256=a4bea7a148a4e50910c8358241fcaa453c0620a8375ba2f8292ba201d570420b
  Stored in directory: /root/.cache/pip/wheels/53/e1/8c/592f08568441aee6d1c9b61bc7a425b509e0607beb306ebf69
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.10.0-cp37-none-any.whl size=60922 sha256=0e246cf712fef18172f2652785cfdaaab56d223f7758635e7c6418f269044ecf
  Stored in directory: /root/.cache/pip/wheels/4a/4e/48/b02ad6dc75235fc4c0742d4e99571fe7d729e60bf365105be4
Successfully built ibm-watson ibm-cloud-sdk-core


In [8]:
####imports for STT
import json
from os.path import join#, dirname
from ibm_watson import SpeechToTextV1
#from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

####imports for evaluation metrics
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [9]:
authenticator = IAMAuthenticator('QMS0quIPO2Dp_YHKBYlTZYHHkOLGRx5QzJn6fg9-2iRJ') 
service = SpeechToTextV1(authenticator = authenticator)
service.set_service_url('https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/8fbf5b1b-3804-4217-aa1c-7ab04c5ad491')

In [10]:
def calculate_wer(reference, hypothesis):
    
    # initialisation
    import numpy
    d = numpy.zeros((len(reference) + 1) * (len(hypothesis) + 1),
                    dtype=numpy.uint8)
    d = d.reshape((len(reference) + 1, len(hypothesis) + 1))
    for i in range(len(reference) + 1):
        for j in range(len(hypothesis) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(reference) + 1):
        for j in range(1, len(hypothesis) + 1):
            if reference[i - 1] == hypothesis[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(reference)][len(hypothesis)] / float(len(reference))

In [11]:
def STT(audio,txt):
    #Hypothesis
    with open(join(audio), 'rb') as audio_file:
        dic = json.loads(json.dumps(service.recognize(audio=audio_file,content_type='audio/mp3',model='en-US_NarrowbandModel',continuous=True).get_result(), indent=2))
    str = ""
    while bool(dic.get('results')):
        str = dic.get('results').pop().get('alternatives').pop().get('transcript')+str[:]  
    hypothesis=str
    
    #reference
    reference = open(txt)
    reference = reference.read().replace('\n', ' ')
    print("Reference; ",reference)
    print("Hypothesis: ",hypothesis)
    
    ##WER##
    print("WER: ",calculate_wer(reference.split(),hypothesis.split()))
    
    ##ROUGE##
    rouge = Rouge()
    print("ROUGE: ", rouge.get_scores(hypothesis, reference)[0]['rouge-l']['r'])
    
    ##BLEU##
    smoothie = SmoothingFunction().method4
    BLEU = sentence_bleu(reference, hypothesis, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    print("BLEU: ",BLEU)

In [12]:
STT("/content/drive/MyDrive/STT Sample files/audio1.mp3","/content/drive/MyDrive/STT Sample files/audio1.txt")

Reference;  President Bok, former President Rudenstine, incoming President Faust, members of the Harvard Corporation and the Board of Overseers, members of the faculty, parents, and especially, the graduates: I’ve been waiting more than 30 years to say this: “Dad, I always told you I’d come back and get my degree.” I want to thank Harvard for this timely honor. I’ll be changing my job next year … and it will be nice to finally have a college degree on my resume. I applaud the graduates today for taking a much more direct route to your degrees. For my part, I’m just happy that the Crimson has called me “Harvard’s most successful dropout.” I guess that makes me valedictorian of my own special class … I did the best of everyone who failed. But I also want to be recognized as the guy who got Steve Ballmer to drop out of business school. I’m a bad influence. That’s why I was invited to speak at your graduation. If I had spoken at your orientation, fewer of you might be here today. Harvard w

In [13]:
STT("/content/drive/MyDrive/STT Sample files/audio2.mp3","/content/drive/MyDrive/STT Sample files/audio2.txt")

Reference;  Hi, everybody. It's Curtis and I'm here with Amy from Real English Conversations. And we're going to talk about driving in Canada and North America. And it all starts with getting your learner's licence, right Amy? Yeah, definitely. In order to be on the road behind the wheel of a vehicle, you've got to have a license. And the first thing that you do is you actually--at least in Canada, we do things in a very specific way. We have something called a graduated licensing program. I'm sure that the United States does it differently. But you pick up this book and it has all the rules of the road. Yeah, you have to study it because you have to write a written test or something like that. Yeah, it's been a long time. Right, Curtis? It's been a very long time for me, yeah. So, yeah, you have to go in. There's a test that makes sure that obviously you've studied the book, you know about the rules of making a right hand turn, for example, when you need to yield, when you need to sto

In [14]:
STT("/content/drive/MyDrive/STT Sample files/audio3.mp3","/content/drive/MyDrive/STT Sample files/audio3.txt")

Reference;  And Abraham Lincoln also was a dreamer and you know, but he said one thing that makes most sense in trying to achieve this goal that I have dreamed for my nation. He said, “If I have six hours to cut down a tree, then I would spend the first four hours sharpening the axe.” There’s a great philosophy in that. In this era of instant gratification we just keep thinking we can achieve all these goals by just tweaking this, tweaking that, it’s not true. I really believe that a missionary zeal is required to make that quantum change, that can make 2030 of what I’m dreaming about right now. And just let’s look at India as a country, what a unique nation! Seriously. Thousands of years old of culture and tradition, many many invasions, being ruled for many years and we still somehow managed to maintain our identity. We still somehow have managed to maintain our Indian-ness — our beliefs, our faith and you know, yeah, there has been — we have our drawbacks — there is corruption, ther

In [15]:
STT("/content/drive/MyDrive/STT Sample files/audio4.mp3","/content/drive/MyDrive/STT Sample files/audio4.txt")

Reference;  Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Hypothesis:  please call Stella ask her to bring the team leader from the store six person so fresh snow peas five P. slash %HESITATION %HESITATION pretty teas and maybe a snack for her brother Bob we always send you to a smaller plastics net and the big toy a flow for the kids she can scoop the being into three Redbacks and every vehicle meet her Wednesday at the train station 
WER:  0.5797101449275363
ROUGE:  0.5178571428571429
BLEU:  0.2520066835834947


In [16]:
STT("/content/drive/MyDrive/STT Sample files/audio5.mp3","/content/drive/MyDrive/STT Sample files/audio5.txt")

Reference;  Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Hypothesis:  please call Stella ask your to bring leave Phoenix who is here from the store six points for fresh snow peas five fleet flames all blue cheese and maybe a snack for her brother Bob we also need a small plastic snake and beak toy for frog for we keep she can scoop leave Phoenix in full so the read back and we will go meet your Wednesday at the train station 
WER:  0.5217391304347826
ROUGE:  0.625
BLEU:  0.24897977631760948


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
